In [1]:
# Load and Split
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser,StrOutputParser

from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.documents import Document

from ragas import EvaluationDataset, RunConfig, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from pydantic import BaseModel, Field

from textwrap import dedent
from operator import itemgetter
from pprint import pprint
import random

from dotenv import load_dotenv
load_dotenv()


True

In [4]:
COLLECTION_NAME = "olympic_info"
DOC_PATH = 'data/olympic.txt'
#  Vector DB와 동일하게 split(위의 것을 사용해도 된다.)
loader = TextLoader(DOC_PATH, encoding='utf-8')
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini", chunk_size=500, chunk_overlap=100
)
docs = loader.load_and_split(splitter) # list[Document]

len(docs)

36

In [5]:
### 평가 데이터로 사용할 context 추출
total_samples = 3

# index shuffle 후 total_samples만큼 context 추출

idx_list = list(range(len(docs)))
random.shuffle(idx_list)

eval_context_list = []
while len(eval_context_list) < total_samples:
    idx = idx_list.pop()
    context = docs[idx].page_content
    if len(context) > 100: # 100글자 이상인 text만 사용
        eval_context_list.append(context)

len(eval_context_list)

3

In [6]:
# user_input: 질문
# ####### retrieved_contexts: 검색된 문서의 내용(page_content)들
# qa_context: 질문 답변 쌍을 만들 때 참고한 context
        # retrieved_contexts: 검색된 문서의 내용은 실제 RAG 실행시 넣는다.
        # response: 모델의 답변 - 실제 RAG 실행시 넣는다.
# reference: 정답
class EvalDatasetSchema(BaseModel):
    user_input: str = Field(..., title="질문(question)")
    qa_context: list[str] = Field(..., title="질문-답변 쌍을 만들 때 참조한 context.")
    reference: str = Field(..., title="질문의 정답(ground truth)")

parser = JsonOutputParser(pydantic_object=EvalDatasetSchema)

eval_model = ChatOpenAI(model="gpt-4o")
prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개 질문-정답 쌍을 생성하세요. 
        
        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [Context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [Context]에 없는 내용을 가지고 질문-정답을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.
        하나의 질문에는 한 가지씩만 내용만 작성합니다. 
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        정답은 반드시 [Context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        질문과 정답을 만들고 그 내용이 [Context] 에 있는 항목인지 다시 한번 확인합니다.
        생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
        질문-답변 쌍은 반드시 {num_questions}개를 만들어야 합니다.

        출력 형식: {format_instructions}
                    
        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
# print(prompt_template.template)

eval_dataset_generator = prompt_template | eval_model | parser

In [7]:
############################################################
# eval_context_list 모두로 만들기
# 
# 생성된 질문-답변을 눈으로 보고 검증한 및 수정해야 한다.
############################################################
eval_data_list = []
num_questions = 5
for context in eval_context_list:
    _eval_data_list = eval_dataset_generator.invoke({"context":context, "num_questions":num_questions})
    eval_data_list.extend(_eval_data_list)

In [8]:
import pandas as pd
eval_df = pd.DataFrame(eval_data_list)
eval_df.shape

(15, 3)

In [10]:
## 생성 된 질문/답 쌍 확인
eval_df.head()
eval_df.tail()

,user_input,qa_context,reference
10,다큐멘터리 '파노라마'가 조사한 주제는 무엇인가?,[BBC 다큐멘터리인 '파노라마'에서는 '매수된 올림픽'이란 주제로 2004년 8월...,'파노라마'는 2012년 하계 올림픽의 개최지 선정과 관련된 뇌물을 조사했다.
11,베르트랑 들라노에가 비난한 인물은 누구인가?,[특히 파리 시장이었던 베르트랑 들라노에(Bertrand Delanoë)는 영국의 ...,베르트랑 들라노에는 영국의 총리인 토니 블레어를 비난했다.
12,자크 시라크 대통령의 반응은 어땠는가?,[그는 당시 프랑스 대통령이었던 자크 시라크를 목격자로 내세웠지만 시라크 대통령은 ...,자크 시라크 대통령은 분쟁에 휘말리는 것을 피하고 인터뷰를 삼갔다.
13,마크 호들러가 주장한 내용은 무엇인가?,[이번에는 스위스 국적의 IOC위원 마크 호들러(Marc Hodler)가 이 논쟁의...,마크 호들러는 토리노가 IOC위원들에게 뇌물수수를 했다고 주장했다.
14,토리노가 개최지로 선정되는데 어떤 영향이 있었는가?,[이 언행이 많은 IOC위원들이 시온에 대해 언짢게 생각하게 되고 토리노가 개최지로...,마크 호들러의 발언은 많은 IOC위원들이 시온에 대해 언짢게 생각하게 하고 토리노가...


# Chain 구성

In [11]:
# Vector Store 연결
COLLECTION_NAME = "olympic_info"
PERSIST_DIRECTORY = "vector_store/olympic_info"
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [12]:
vector_store._collection.count()

36

In [13]:
# Chain 구성
# prompt template. langchain hub에 등록된 것을 가져와서 사용.
prompt_template = hub.pull("rlm/rag-prompt")
# prompt_template

# Retriever 생성
retriever = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k':3,
        'fetch_k':10,
        'lambda_mult':0.5
    }
)
model = ChatOpenAI(model="gpt-4o-mini")

def format_docs(src_docs:dict[str, list[Document]]) -> str:
    """list[Document]: Vector Store에서 검색한 context들에서 
    page_content만 추출해서 하나의 문자열로 합쳐서 반환"""
    docs = src_docs['context']
    return "\n\n".join([doc.page_content for doc in docs])

def str_from_documents(docs: list[Document]) -> list[str]:
    """list[Document]에서 page_content 값들만 추출한 list를 반환."""
    return [doc.page_content for doc in docs]

rag_chain = (
    RunnablePassthrough() # rag chain을 RunnableSequence로 만들기 위해 Runnable인 것으로 시작.
    | {
        "context": retriever, "question":RunnablePassthrough()
    } # retriver -> {"context":list[Document], "question":"user input"}
    | {
        # 앞에서 넘어온 dictionary에서 context(List[Document])를 추출 -> page_content값들을 list로 반환. list[str]
        "source_context" : itemgetter("context") | RunnableLambda(str_from_documents), 
        "llm_answer": {
            # {"context":list[Document]} -> str(page_content들만 모은 string)
            "context": RunnableLambda(format_docs), "question":itemgetter("question")
        } | prompt_template | model | StrOutputParser()  # LLM 응답 처리 chain. 
    }
)

In [15]:
eval_df['user_input']

0                    동계 올림픽은 몇 년 주기로 개최되었나요?
1                       1회 동계 올림픽은 어디서 열렸나요?
2         피겨스케이팅과 아이스하키는 언제 하계 올림픽 종목이 되었나요?
3                 동계 올림픽은 언제 처음 열리기로 결정되었나요?
4                1994년 동계 올림픽은 어느 나라에서 열렸나요?
5     1936년 하계 올림픽에서 나치 독일이 보여주려 했던 것은 무엇인가?
6        제시 오언스가 1936년 하계 올림픽에서 획득한 금메달 개수는?
7                소련이 처음으로 참가한 올림픽은 어떤 대회였는가?
8                    노동자 올림픽은 어떤 이유로 조직되었는가?
9                소련이 스포츠 강국으로서의 면모를 드러낸 시기는?
10               다큐멘터리 '파노라마'가 조사한 주제는 무엇인가?
11                  베르트랑 들라노에가 비난한 인물은 누구인가?
12                     자크 시라크 대통령의 반응은 어땠는가?
13                     마크 호들러가 주장한 내용은 무엇인가?
14              토리노가 개최지로 선정되는데 어떤 영향이 있었는가?
Name: user_input, dtype: object

In [14]:
# rag_chain에 평가 질문을 입력해서 context들과 모델답변을 응답 받아 eval_dataset(eval_df)에 추가.
context_list = []
response_list = []

for user_input in eval_df['user_input']:
    res = rag_chain.invoke(user_input)
    context_list.append(res['source_context'])
    response_list.append(res['llm_answer'])

In [16]:
print(len(context_list), len(response_list))
# pprint(context_list[:2])
# pprint(response_list[:2])

15 15


In [18]:
response_list

['동계 올림픽은 4년 주기로 개최됩니다. 이 전통은 1924년 첫 동계 올림픽부터 시작되었으며, 1994년부터는 하계 올림픽이 끝난 후 2년 뒤에 개최되고 있습니다.',
 '1회 동계 올림픽은 1924년 프랑스의 샤모니에서 열렸습니다. 대회는 11일간 진행되었고, 총 16개 종목의 경기가 치러졌습니다.',
 '피겨스케이팅과 아이스하키는 하계 올림픽 종목이 아닙니다. 이 두 스포츠는 동계 올림픽에서 정식 종목으로 채택되었습니다. 따라서 하계 올림픽에서는 이들 스포츠의 경기가 이루어지지 않습니다.',
 '동계 올림픽은 1921년 로잔에서 열린 올림픽 의회에서 열기로 합의되었습니다. 첫 번째 동계 올림픽은 1924년 프랑스의 샤모니에서 11일간 진행되었습니다. 이 대회는 IOC가 하계 올림픽과 같은 해에 4년 주기로 개최하기로 결정한 것입니다.',
 '1994년 동계 올림픽은 노르웨이의 릴레함메르에서 열렸습니다.',
 '1936년 하계 올림픽에서 나치 독일은 자비롭고 평화를 위한다는 이미지를 보여주고 싶어 했습니다. 또한 아리안족의 우월성을 드러내려 했으나, 흑인 선수 제시 오언스가 금메달 4개를 따내면서 그 목표는 이루어지지 않았습니다.',
 '제시 오언스는 1936년 하계 올림픽에서 금메달을 4개 획득했습니다. 이는 나치 독일의 아리안족 우월성을 반박하는 상징적 사건이었습니다.',
 '소련이 처음으로 참가한 올림픽은 1952년 헬싱키에서 열린 하계 올림픽입니다. 그 이전에는 소련이 스파르타키아다라는 대회에 참가하였지만, 올림픽에는 처음으로 참석한 것입니다. 이후 소련은 1956년부터 1988년까지 올림픽에서 두각을 나타냈습니다.',
 '노동자 올림픽은 올림픽이 자본가와 귀족들의 대회로 여겨지면서 그에 대한 대안으로 조직된 대회입니다. 이는 1920년대와 1930년대 전쟁 기간 사이에 다른 공산주의 국가들과 함께 개최되었습니다. 따라서 노동자 올림픽은 사회주의 이념을 바탕으로 한 스포츠 행사로 시작되었습니다.',
 '소련은 1952년 헬싱키 하계 올림픽에 처음 

In [19]:
eval_df["retrieved_contexts"] = context_list # context 추가
eval_df["response"] = response_list   # 정답 추가

In [20]:
eval_df.head(3)

,user_input,qa_context,reference,retrieved_contexts,response
0,동계 올림픽은 몇 년 주기로 개최되었나요?,[동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 ...,"동계 올림픽은 원래 4년 주기로 하계 올림픽과 같은 해에 열렸으나, 1994년부터는...",[동계올림픽\n동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계...,동계 올림픽은 4년 주기로 개최됩니다. 이 전통은 1924년 첫 동계 올림픽부터 시...
1,1회 동계 올림픽은 어디서 열렸나요?,"[1회 동계올림픽은 1924년, 프랑스의 샤모니에서 11일간 진행되었고, 16개 종...",1회 동계 올림픽은 프랑스의 샤모니에서 열렸습니다.,[동계올림픽\n동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계...,1회 동계 올림픽은 1924년 프랑스의 샤모니에서 열렸습니다. 대회는 11일간 진행...
2,피겨스케이팅과 아이스하키는 언제 하계 올림픽 종목이 되었나요?,"[피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어...","피겨스케이팅은 1908년에, 아이스하키는 1920년에 하계 올림픽 종목이 되었습니다.",[하계올림픽\n1859년 자파스 올림픽에 참가한 선수의 수는 250명을 넘지 못했다...,피겨스케이팅과 아이스하키는 하계 올림픽 종목이 아닙니다. 이 두 스포츠는 동계 올림...


# 평가

In [21]:
# Dataframe으로 부터 EvalDataset 생성
eval_dataset = EvaluationDataset.from_pandas(eval_df)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=15)

In [22]:
# HuggingFace에 업로드 -> datasets.Dataset 으로 변환
eval_dataset.to_hf_dataset()#.push_to_hub()

Dataset({
    features: ['user_input', 'retrieved_contexts', 'response', 'reference'],
    num_rows: 15
})

In [23]:
# model_name = "gpt-4o"
model_name = "gpt-4o-mini"
model = ChatOpenAI(model=model_name)
eval_llm = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)


## GPT-4o-mini 모델을 사용하여 평가 
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]
result = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [24]:
result

{'context_recall': 0.8000, 'llm_context_precision_with_reference': 0.7556, 'faithfulness': 0.8389, 'answer_relevancy': 0.3016}

In [ ]:
# gpt-4o : 분당 토큰 리미트에 걸려 RateLimitError 가 발생할 수있다. gpt-40: 30,000 TPM, gpt-4o-mini: 200,000 TPM
# 또한 network 연결등 문제가 발생하면 timeout 이 되어 평가가 실패할 수 있다.
# LLM 연결과 관련해 timetout이나 ratelimiterror 발생시 metrics를 나눠서 실행, 설정 변경을 통해 해결한다.
## https://platform.openai.com/settings/organization/limits
run_config = RunConfig(
    timeout=360,     # LLM 호출 이후 최대 대기 시간. 지정한 초까지 응답을 기다린다. 
    max_retries=20, # API 호출시 지정한 횟수만큼 재시도 한다.
    max_wait=360,   # 재시도 대기 시간(초) 180초 기다린 후 재시도 한다.
    max_workers=1   # 병렬처리 worker 수. 1로 설정하면 순차적으로 처리한다. (default: 16)
)
metrics1 = [
    LLMContextRecall(llm=eval_llm),
]
metrics2 = [
    LLMContextPrecisionWithReference(llm=eval_llm),
]
metrics3 = [
    Faithfulness(llm=eval_llm),
]
metrics4 = [
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]

In [ ]:
result1 = evaluate(dataset=eval_dataset, metrics=metrics1, run_config=run_config)
result1

In [ ]:
result2 = evaluate(dataset=eval_dataset, metrics=metrics2, run_config=run_config)
result2

In [ ]:
result3 = evaluate(dataset=eval_dataset, metrics=metrics3, run_config=run_config)
result3

In [ ]:
result4 = evaluate(dataset=eval_dataset, metrics=metrics4, run_config=run_config)
result4